In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

#### compare

In [4]:
# baseline = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_small_business_busshellv31_air231_results_with1rep_new2w20230821-094133.csv", 
#                         verbose = False, low_memory = False, encoding = 'iso-8859-1')
# after = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_small_business_busshellv31_air231_results_with3rep_new2w20230821-102310.csv", 
#                      verbose = False, low_memory = False, encoding = 'iso-8859-1')
from dsgtools import azure
baseline = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_data/1010_small_business_busshellv31_air231_results_with1rep_w20231005-103509.csv", 
                        verbose = False, low_memory = False, encoding = 'iso-8859-1')
after = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_data/1010_small_business_busshellv31_air231_results_with3rep_new2w20231005-110300.csv", 
                     verbose = False, low_memory = False, encoding = 'iso-8859-1')
print(baseline.shape)
print(after.shape)

(9994, 2304)
(9995, 2304)


In [5]:
from dsgtools.reporting import ImpactAnalysisReport
exclude = ['accountnumber', 'historydateyyyymm', 'bus_company_name', 'powid', 'proxid', 'seleid', 'orgid', 'ultid', 'lnlexidsele','model1name', 'model1score',
           'model1rc1', 'model1rc2', 'model1rc3', 'model1rc4', 'model1rc5', 'model1rc6', 'model2name', 'model2score', 'model2rc1', 'model2rc2', 'model2rc3', 'model2rc4',
           'model2rc5', 'model2rc6', 'model3name', 'model3score', 'model3rc1', 'model3rc2', 'model3rc3', 'model3rc4', 'model3rc5', 'model3rc6', 'model4name', 'model4score',
           'model4rc1', 'model4rc2', 'model4rc3', 'model4rc4', 'model4rc5', 'model4rc6', 'model5name', 'model5score', 'model5rc1', 'model5rc2', 'model5rc3', 'model5rc4',
           'model5rc5', 'model5rc6', 'model6name', 'model6score', 'model6rc1', 'model6rc2', 'model6rc3', 'model6rc4', 'model6rc5', 'model6rc6', 'model7name', 'model7score',
           'model7rc1', 'model7rc2', 'model7rc3', 'model7rc4', 'model7rc5', 'model7rc6', 'model8name', 'model8score', 'model8rc1', 'model8rc2', 'model8rc3', 'model8rc4',
           'model8rc5', 'model8rc6', 'model9name', 'model9score', 'model9rc1', 'model9rc2', 'model9rc3', 'model9rc4', 'model9rc5', 'model9rc6', 'model10name', 'model10score',
           'model10rc1', 'model10rc2', 'model10rc3', 'model10rc4', 'model10rc5', 'model10rc6', 'errorcode', 'time_ms']
sbfe_attr = [x for x in baseline.columns if "sbfe" in x]
b2b = [x for x in baseline.columns if "b2b" in x]
rv_attr = [x for x in baseline.columns if x not in sbfe_attr and x not in exclude]
model = ["model1score", "model2score"]

exception_dict = dict()
for x in baseline.columns:
    if x in sbfe_attr:
        exception_dict[x] = [-99, -98, -97]
    if x in b2b:
        exception_dict[x] = [-99999, -99998, -99997]
    if x in rv_attr:
        exception_dict[x] = [-1]
    if x in model:
        exception_dict[x] = [0, 100, 200, 222]
        
cr = ImpactAnalysisReport(baseline, after, keys="accountnumber",
           exceptions=exception_dict,
           dataframe_names = ("baseline", "after"))
cr.to_xlsx("./_temp/comparison_attr.xlsx", overwrite= True)

In [7]:
temp_before = baseline[field].copy()
temp_after = after[field].copy()
sample = pd.read_adls('Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/input_sample_DS.parquet') #AccountNumber
sample.unique_id = sample.unique_id.astype(str)
temp_before = temp_before.merge(temp_after, on = "accountnumber")
temp_before = temp_before.merge(sample[["unique_id", "group"]], left_on = "accountnumber", right_on = "unique_id")
print(temp_before.shape)
temp_before = temp_before.fillna("")

(9987, 131)


In [9]:
col_2 = [x for x in temp_before.columns if ("_flag" in x) & ("rep2" in x)] ## rep dob is provided as 0 somehow
col_3 = [x for x in temp_before.columns if ("_flag" in x) & ("rep3" in x)]
print(len(col_2))
print(len(col_3))
temp_before['sum_diff_rep2']=temp_before[col_2]. sum(axis=1)
temp_before['sum_diff_rep3']=temp_before[col_3]. sum(axis=1)

28
28


In [10]:
fmt = make_format(cuts = [-np.inf, 0, np.inf])

freq("group", 'seleid_x', "errorcode_x", "sum_diff_rep3", df = temp_before, format = [None, fmt, None, fmt], cross = False, observed = True)

Count  \
group seleid_x errorcode_x                                        sum_diff_rep3           
3rep  <= 0                                                        1+             1200.0   
               0 ReceivedRoxieException: (Please input the min... <= 0              2.0   
      1+                                                          1+             5786.0   
2rep  <= 0                                                        <= 0            257.0   
               0 ReceivedRoxieException: (Please input the min... <= 0              1.0   
      1+                                                          <= 0            741.0   
1rep  <= 0                                                        <= 0            132.0   
               0 ReceivedRoxieException: (Please input the min... <= 0             13.0   
      1+                                                          <= 0            855.0   
0rep  <= 0     0 ReceivedRoxieException: (Please input the min... <= 0            921.0   
               0 ReceivedRoxieException: (Please input the min... <= 0             79.0   

                                                                                      Pct  \
group seleid_x errorcode_x                                        sum_diff_rep3             
3rep  <= 0                                                        1+             0.120156   
               0 ReceivedRoxieException: (Please input the min... <= 0           0.000200   
      1+                                                          1+             0.579353   
2rep  <= 0                                                        <= 0           0.025733   
               0 ReceivedRoxieException: (Please input the min... <= 0           0.000100   
      1+                                                          <= 0           0.074196   
1rep  <= 0                                                        <= 0           0.013217   
               0 ReceivedRoxieException: (Please input the min... <= 0           0.001302   
      1+                                                          <= 0           0.085611   
0rep  <= 0     0 ReceivedRoxieException: (Please input the min... <= 0           0.092220   
               0 ReceivedRoxieException: (Please input the min... <= 0           0.007910   

                                                                                 Cuml Count  \
group seleid_x errorcode_x                                        sum_diff_rep3               
3rep  <= 0                                                        1+                 1200.0   
               0 ReceivedRoxieException: (Please input the min... <= 0               1202.0   
      1+                                                          1+                 6988.0   
2rep  <= 0                                                        <= 0               7245.0   
               0 ReceivedRoxieException: (Please input the min... <= 0               7246.0   
      1+                                                          <= 0               7987.0   
1rep  <= 0                                                        <= 0               8119.0   
               0 ReceivedRoxieException: (Please input the min... <= 0               8132.0   
      1+                                                          <= 0               8987.0   
0rep  <= 0     0 ReceivedRoxieException: (Please input the min... <= 0               9908.0   
               0 ReceivedRoxieException: (Please input the min... <= 0               9987.0   

                                                                                 Cuml Pct  
group seleid_x errorcode_x                                        sum_diff_rep3            
3rep  <= 0                                                        1+             0.120156  
               0 ReceivedRoxieException: (Please input the min... <= 0           0.120356  
      1+                                                          1+             0.6

In [11]:
fmt = make_format(cuts = [-np.inf, 0, np.inf])
freq("group", 'seleid_x', "errorcode_x", "sum_diff_rep2", df = temp_before, format = [None, fmt, None, fmt], cross = False, observed = True)

Count  \
group seleid_x errorcode_x                                        sum_diff_rep2           
3rep  <= 0                                                        1+             1200.0   
               0 ReceivedRoxieException: (Please input the min... <= 0              2.0   
      1+                                                          1+             5786.0   
2rep  <= 0                                                        1+              257.0   
               0 ReceivedRoxieException: (Please input the min... <= 0              1.0   
      1+                                                          1+              741.0   
1rep  <= 0                                                        <= 0            132.0   
               0 ReceivedRoxieException: (Please input the min... <= 0             13.0   
      1+                                                          <= 0            855.0   
0rep  <= 0     0 ReceivedRoxieException: (Please input the min... <= 0            921.0   
               0 ReceivedRoxieException: (Please input the min... <= 0             79.0   

                                                                                      Pct  \
group seleid_x errorcode_x                                        sum_diff_rep2             
3rep  <= 0                                                        1+             0.120156   
               0 ReceivedRoxieException: (Please input the min... <= 0           0.000200   
      1+                                                          1+             0.579353   
2rep  <= 0                                                        1+             0.025733   
               0 ReceivedRoxieException: (Please input the min... <= 0           0.000100   
      1+                                                          1+             0.074196   
1rep  <= 0                                                        <= 0           0.013217   
               0 ReceivedRoxieException: (Please input the min... <= 0           0.001302   
      1+                                                          <= 0           0.085611   
0rep  <= 0     0 ReceivedRoxieException: (Please input the min... <= 0           0.092220   
               0 ReceivedRoxieException: (Please input the min... <= 0           0.007910   

                                                                                 Cuml Count  \
group seleid_x errorcode_x                                        sum_diff_rep2               
3rep  <= 0                                                        1+                 1200.0   
               0 ReceivedRoxieException: (Please input the min... <= 0               1202.0   
      1+                                                          1+                 6988.0   
2rep  <= 0                                                        1+                 7245.0   
               0 ReceivedRoxieException: (Please input the min... <= 0               7246.0   
      1+                                                          1+                 7987.0   
1rep  <= 0                                                        <= 0               8119.0   
               0 ReceivedRoxieException: (Please input the min... <= 0               8132.0   
      1+                                                          <= 0               8987.0   
0rep  <= 0     0 ReceivedRoxieException: (Please input the min... <= 0               9908.0   
               0 ReceivedRoxieException: (Please input the min... <= 0               9987.0   

                                                                                 Cuml Pct  
group seleid_x errorcode_x                                        sum_diff_rep2            
3rep  <= 0                                                        1+             0.120156  
               0 ReceivedRoxieException: (Please input the min... <= 0           0.120356  
      1+                                                          1+             0.6

In [8]:
temp_before['inputcheckauthrep2lastname_flag'] = np.where(temp_before.inputcheckauthrep2lastname_x != temp_before.inputcheckauthrep2lastname_y, 1, 0)
temp_before['inputcheckauthrep2firstname_flag'] = np.where(temp_before.inputcheckauthrep2firstname_x != temp_before.inputcheckauthrep2firstname_y, 1, 0)
temp_before['inputcheckauthrep2addr_flag'] = np.where(temp_before.inputcheckauthrep2addr_x != temp_before.inputcheckauthrep2addr_y, 1, 0)
temp_before['inputcheckauthrep2city_flag'] = np.where(temp_before.inputcheckauthrep2city_x != temp_before.inputcheckauthrep2city_y, 1, 0)
temp_before['inputcheckauthrep2zip_flag'] = np.where(temp_before.inputcheckauthrep2zip_x != temp_before.inputcheckauthrep2zip_y, 1, 0)
temp_before['inputcheckauthrep2ssn_flag'] = np.where(temp_before.inputcheckauthrep2ssn_x != temp_before.inputcheckauthrep2ssn_y, 1, 0)
temp_before['inputcheckauthrep2state_flag'] = np.where(temp_before.inputcheckauthrep2state_x != temp_before.inputcheckauthrep2state_y, 1, 0)
temp_before['inputcheckauthrep2phone_flag'] = np.where(temp_before.inputcheckauthrep2phone_x != temp_before.inputcheckauthrep2phone_y, 1, 0)
temp_before['inputcheckauthrep3lastname_flag'] = np.where(temp_before.inputcheckauthrep3lastname_x != temp_before.inputcheckauthrep3lastname_y, 1, 0)
temp_before['inputcheckauthrep3firstname_flag'] = np.where(temp_before.inputcheckauthrep3firstname_x != temp_before.inputcheckauthrep3firstname_y, 1, 0)
temp_before['inputcheckauthrep3addr_flag'] = np.where(temp_before.inputcheckauthrep3addr_x != temp_before.inputcheckauthrep3addr_y, 1, 0)
temp_before['inputcheckauthrep3city_flag'] = np.where(temp_before.inputcheckauthrep3city_x != temp_before.inputcheckauthrep3city_y, 1, 0)
temp_before['inputcheckauthrep3state_flag'] = np.where(temp_before.inputcheckauthrep3state_x != temp_before.inputcheckauthrep3state_y, 1, 0)
temp_before['inputcheckauthrep3zip_flag'] = np.where(temp_before.inputcheckauthrep3zip_x != temp_before.inputcheckauthrep3zip_y, 1, 0)
temp_before['inputcheckauthrep3phone_flag'] = np.where(temp_before.inputcheckauthrep3phone_x != temp_before.inputcheckauthrep3phone_y, 1, 0)
temp_before['busexeclinkinqoverlapcount2_flag'] = np.where(temp_before.busexeclinkinqoverlapcount2_x != temp_before.busexeclinkinqoverlapcount2_y, 1, 0)
temp_before['inputcheckauthrep3ssn_flag'] = np.where(temp_before.inputcheckauthrep3ssn_x != temp_before.inputcheckauthrep3ssn_y, 1, 0)
temp_before['busexeclinkutiloverlapcount2_flag'] = np.where(temp_before.busexeclinkutiloverlapcount2_x != temp_before.busexeclinkutiloverlapcount2_y, 1, 0)
temp_before['busexeclinkrep2nameonfile_flag'] = np.where(temp_before.busexeclinkrep2nameonfile_x != temp_before.busexeclinkrep2nameonfile_y, 1, 0)
temp_before['busexeclinkbusnamerep2full_flag'] = np.where(temp_before.busexeclinkbusnamerep2full_x != temp_before.busexeclinkbusnamerep2full_y, 1, 0)
temp_before['busexeclinkpropoverlapcount2_flag'] = np.where(temp_before.busexeclinkpropoverlapcount2_x != temp_before.busexeclinkpropoverlapcount2_y, 1, 0)
temp_before['busexeclinkbusnamerep2first_flag'] = np.where(temp_before.busexeclinkbusnamerep2first_x != temp_before.busexeclinkbusnamerep2first_y, 1, 0)
temp_before['busexeclinkbusnamerep2last_flag'] = np.where(temp_before.busexeclinkbusnamerep2last_x != temp_before.busexeclinkbusnamerep2last_y, 1, 0)
temp_before['busexeclinkrep2addrbusaddr_flag'] = np.where(temp_before.busexeclinkrep2addrbusaddr_x != temp_before.busexeclinkrep2addrbusaddr_y, 1, 0)
temp_before['busexeclinkrep2addronfile_flag'] = np.where(temp_before.busexeclinkrep2addronfile_x != temp_before.busexeclinkrep2addronfile_y, 1, 0)
temp_before['busexeclinkbusaddrrep2owned_flag'] = np.where(temp_before.busexeclinkbusaddrrep2owned_x != temp_before.busexeclinkbusaddrrep2owned_y, 1, 0)
temp_before['busexeclinkrep2ssnonfile_flag'] = np.where(temp_before.busexeclinkrep2ssnonfile_x != temp_before.busexeclinkrep2ssnonfile_y, 1, 0)
temp_before['busexeclinkrep2ssnbustin_flag'] = np.where(temp_before.busexeclinkrep2ssnbustin_x != temp_before.busexeclinkrep2ssnbustin_y, 1, 0)
temp_before['busexeclinkinqoverlapcount3_flag'] = np.where(temp_before.busexeclinkinqoverlapcount3_x != temp_before.busexeclinkinqoverlapcount3_y, 1, 0)
temp_before['busexeclinkbusnamerep3first_flag'] = np.where(temp_before.busexeclinkbusnamerep3first_x != temp_before.busexeclinkbusnamerep3first_y, 1, 0)
temp_before['busexeclinkrep3nameonfile_flag'] = np.where(temp_before.busexeclinkrep3nameonfile_x != temp_before.busexeclinkrep3nameonfile_y, 1, 0)
temp_before['busexeclinkbusnamerep3full_flag'] = np.where(temp_before.busexeclinkbusnamerep3full_x != temp_before.busexeclinkbusnamerep3full_y, 1, 0)
temp_before['busexeclinkbusnamerep3last_flag'] = np.where(temp_before.busexeclinkbusnamerep3last_x != temp_before.busexeclinkbusnamerep3last_y, 1, 0)
temp_before['busexeclinkutiloverlapcount3_flag'] = np.where(temp_before.busexeclinkutiloverlapcount3_x != temp_before.busexeclinkutiloverlapcount3_y, 1, 0)
temp_before['busexeclinkpropoverlapcount3_flag'] = np.where(temp_before.busexeclinkpropoverlapcount3_x != temp_before.busexeclinkpropoverlapcount3_y, 1, 0)
temp_before['busexeclinkrep3addrbusaddr_flag'] = np.where(temp_before.busexeclinkrep3addrbusaddr_x != temp_before.busexeclinkrep3addrbusaddr_y, 1, 0)
temp_before['busexeclinkrep3addronfile_flag'] = np.where(temp_before.busexeclinkrep3addronfile_x != temp_before.busexeclinkrep3addronfile_y, 1, 0)
temp_before['busexeclinkrep2phoneonfile_flag'] = np.where(temp_before.busexeclinkrep2phoneonfile_x != temp_before.busexeclinkrep2phoneonfile_y, 1, 0)
temp_before['busexeclinkrep2phonebusphone_flag'] = np.where(temp_before.busexeclinkrep2phonebusphone_x != temp_before.busexeclinkrep2phonebusphone_y, 1, 0)
temp_before['busexeclinkbusaddrrep3owned_flag'] = np.where(temp_before.busexeclinkbusaddrrep3owned_x != temp_before.busexeclinkbusaddrrep3owned_y, 1, 0)
temp_before['busexeclinkrep3phonebusphone_flag'] = np.where(temp_before.busexeclinkrep3phonebusphone_x != temp_before.busexeclinkrep3phonebusphone_y, 1, 0)
temp_before['busexeclinkrep3phoneonfile_flag'] = np.where(temp_before.busexeclinkrep3phoneonfile_x != temp_before.busexeclinkrep3phoneonfile_y, 1, 0)
temp_before['busexeclinkrep3ssnbustin_flag'] = np.where(temp_before.busexeclinkrep3ssnbustin_x != temp_before.busexeclinkrep3ssnbustin_y, 1, 0)
temp_before['busexeclinkrep3ssnonfile_flag'] = np.where(temp_before.busexeclinkrep3ssnonfile_x != temp_before.busexeclinkrep3ssnonfile_y, 1, 0)
temp_before['sbfebusexeclinkrep2name_flag'] = np.where(temp_before.sbfebusexeclinkrep2name_x != temp_before.sbfebusexeclinkrep2name_y, 1, 0)
temp_before['sbfebusexeclinkrep2addr_flag'] = np.where(temp_before.sbfebusexeclinkrep2addr_x != temp_before.sbfebusexeclinkrep2addr_y, 1, 0)
temp_before['sbfebusexeclinkrep2ssn_flag'] = np.where(temp_before.sbfebusexeclinkrep2ssn_x != temp_before.sbfebusexeclinkrep2ssn_y, 1, 0)
temp_before['inputcheckauthrep2dob_flag'] = np.where(temp_before.inputcheckauthrep2dob_x != temp_before.inputcheckauthrep2dob_y, 1, 0)
temp_before['sbfebusexeclinkrep3name_flag'] = np.where(temp_before.sbfebusexeclinkrep3name_x != temp_before.sbfebusexeclinkrep3name_y, 1, 0)
temp_before['sbfebusexeclinkrep3addr_flag'] = np.where(temp_before.sbfebusexeclinkrep3addr_x != temp_before.sbfebusexeclinkrep3addr_y, 1, 0)
temp_before['sbfebusexeclinkrep2phone_flag'] = np.where(temp_before.sbfebusexeclinkrep2phone_x != temp_before.sbfebusexeclinkrep2phone_y, 1, 0)
temp_before['sbfebusexeclinkrep3phone_flag'] = np.where(temp_before.sbfebusexeclinkrep3phone_x != temp_before.sbfebusexeclinkrep3phone_y, 1, 0)
temp_before['sbfebusexeclinkrep3ssn_flag'] = np.where(temp_before.sbfebusexeclinkrep3ssn_x != temp_before.sbfebusexeclinkrep3ssn_y, 1, 0)
temp_before['inputcheckauthrep3dob_flag'] = np.where(temp_before.inputcheckauthrep3dob_x != temp_before.inputcheckauthrep3dob_y, 1, 0)
temp_before['inputcheckauthrep2middlename_flag'] = np.where(temp_before.inputcheckauthrep2middlename_x != temp_before.inputcheckauthrep2middlename_y, 1, 0)
temp_before['inputcheckauthrep3middlename_flag'] = np.where(temp_before.inputcheckauthrep3middlename_x != temp_before.inputcheckauthrep3middlename_y, 1, 0)
temp_before['inputcheckauthrep2dl_flag'] = np.where(temp_before.inputcheckauthrep2dl_x != temp_before.inputcheckauthrep2dl_y, 1, 0)
temp_before['inputcheckauthrep3dl_flag'] = np.where(temp_before.inputcheckauthrep3dl_x != temp_before.inputcheckauthrep3dl_y, 1, 0)
temp_before['inputcheckauthrep2dlstate_flag'] = np.where(temp_before.inputcheckauthrep2dlstate_x != temp_before.inputcheckauthrep2dlstate_y, 1, 0)
temp_before['inputcheckauthrep3dlstate_flag'] = np.where(temp_before.inputcheckauthrep3dlstate_x != temp_before.inputcheckauthrep3dlstate_y, 1, 0)
temp_before['inputcheckauthrep2age_flag'] = np.where(temp_before.inputcheckauthrep2age_x != temp_before.inputcheckauthrep2age_y, 1, 0)
temp_before['inputcheckauthrep3age_flag'] = np.where(temp_before.inputcheckauthrep3age_x != temp_before.inputcheckauthrep3age_y, 1, 0)


/tmp/ipykernel_77667/4025077317.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_before['inputcheckauthrep2lastname_flag'] = np.where(temp_before.inputcheckauthrep2lastname_x != temp_before.inputcheckauthrep2lastname_y, 1, 0)
/tmp/ipykernel_77667/4025077317.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_before['inputcheckauthrep2firstname_flag'] = np.where(temp_before.inputcheckauthrep2firstname_x != temp_before.inputcheckauthrep2firstname_y, 1, 0)
/tmp/ipykernel_77667/4025077317.py:3: PerformanceWarning: DataF

In [6]:
field = ["accountnumber", "seleid", 'errorcode', 'inputcheckauthrep2lastname',
'inputcheckauthrep2firstname',
'inputcheckauthrep2addr',
'inputcheckauthrep2city',
'inputcheckauthrep2zip',
'inputcheckauthrep2ssn',
'inputcheckauthrep2state',
'inputcheckauthrep2phone',
'inputcheckauthrep3lastname',
'inputcheckauthrep3firstname',
'inputcheckauthrep3addr',
'inputcheckauthrep3city',
'inputcheckauthrep3state',
'inputcheckauthrep3zip',
'inputcheckauthrep3phone',
'busexeclinkinqoverlapcount2',
'inputcheckauthrep3ssn',
'busexeclinkutiloverlapcount2',
'busexeclinkrep2nameonfile',
'busexeclinkbusnamerep2full',
'busexeclinkpropoverlapcount2',
'busexeclinkbusnamerep2first',
'busexeclinkbusnamerep2last',
'busexeclinkrep2addrbusaddr',
'busexeclinkrep2addronfile',
'busexeclinkbusaddrrep2owned',
'busexeclinkrep2ssnonfile',
'busexeclinkrep2ssnbustin',
'busexeclinkinqoverlapcount3',
'busexeclinkbusnamerep3first',
'busexeclinkrep3nameonfile',
'busexeclinkbusnamerep3full',
'busexeclinkbusnamerep3last',
'busexeclinkutiloverlapcount3',
'busexeclinkpropoverlapcount3',
'busexeclinkrep3addrbusaddr',
'busexeclinkrep3addronfile',
'busexeclinkrep2phoneonfile',
'busexeclinkrep2phonebusphone',
'busexeclinkbusaddrrep3owned',
'busexeclinkrep3phonebusphone',
'busexeclinkrep3phoneonfile',
'busexeclinkrep3ssnbustin',
'busexeclinkrep3ssnonfile',
'sbfebusexeclinkrep2name',
'sbfebusexeclinkrep2addr',
'sbfebusexeclinkrep2ssn',
'inputcheckauthrep2dob',
'sbfebusexeclinkrep3name',
'sbfebusexeclinkrep3addr',
'sbfebusexeclinkrep2phone',
'sbfebusexeclinkrep3phone',
'sbfebusexeclinkrep3ssn',
'inputcheckauthrep3dob',
'inputcheckauthrep2middlename',
'inputcheckauthrep3middlename',
'inputcheckauthrep2dl',
'inputcheckauthrep3dl',
'inputcheckauthrep2dlstate',
'inputcheckauthrep3dlstate',
'inputcheckauthrep2age',
'inputcheckauthrep3age',
]